## 1

Here’s a **neat, structured set of notes** summarizing everything you’ve learned and implemented so far with Pydantic:

---

### Pydantic Notes

---

### 1. BaseModel

* All Pydantic models inherit from `BaseModel`.
* It provides **automatic validation, type enforcement, and serialization**.
* Example:

```python
from pydantic import BaseModel, EmailStr

class Patient(BaseModel):
    name: str
    email: EmailStr
    age: int
```

---

### 2. Field Validator (`field_validator`)

* Used to validate **individual fields**.
* Runs **before or after** the field value is set.
* Useful for **custom rules on a single field**.

```python
from pydantic import field_validator

class Patient(BaseModel):
    age: int

    @field_validator('age')
    def check_age(cls, v):
        if v < 0:
            raise ValueError("Age cannot be negative")
        return v
```

---

### 3. Model Validator (`model_validator`)

* Used to validate **entire model** after all fields are set.
* Allows validation rules that depend on **multiple fields**.

```python
from pydantic import model_validator

class Patient(BaseModel):
    age: int
    contact_details: dict

    @model_validator(mode='after')
    def check_emergency_contact(self):
        if self.age > 60 and 'emergency' not in self.contact_details:
            raise ValueError("Patients older than 60 must have an emergency contact")
        return self
```

---

### 4. Computed Fields (`@computed_field`)

* Used to define **dynamic properties** based on other fields.
* Behaves like a **real field** in Pydantic: included in `.model_dump()` and API responses.
* Always combined with `@property`.

```python
from pydantic import computed_field

class Patient(BaseModel):
    weight: float
    height: float

    @computed_field
    @property
    def bmi(self) -> float:
        return round(self.weight / (self.height ** 2), 2)
```

---

### 5. Nested Models

* Use **models inside other models** for structured and reusable data.
* Improves **readability, organization, and automatic validation**.

```python
class Address(BaseModel):
    city: str
    state: str
    pin: str

class Patient(BaseModel):
    name: str
    age: int
    address: Address
```

* Pydantic validates nested fields automatically.
* Supports passing **dictionaries** or **model instances**.

---

### 6. Serialization (`model_dump`, `model_dump_json`)

* Convert model instances to **dictionaries or JSON**.
* Can include/exclude fields, or control nested model output.

```python
patient_dict = patient.model_dump()
json_data = patient.model_dump_json()
```

* Computed fields and nested models are included automatically if defined with `@computed_field` or nested BaseModel.

---

### 7. Summary of Features

| Feature         | Purpose                                                         |
| --------------- | --------------------------------------------------------------- |
| BaseModel       | Base class providing validation, parsing, and serialization     |
| Field Validator | Validate individual fields                                      |
| Model Validator | Validate multiple fields together                               |
| Computed Field  | Create derived attributes that behave like real fields          |
| Nested Models   | Organize related data hierarchically and validate automatically |
| Serialization   | Convert models to dict/JSON for storage or API responses        |

---

If you want, I can make a **visual flow diagram** showing **how all these features interact during object creation and serialization**, which is super handy for notes.

Do you want me to make that?


In [1]:
from pydantic import BaseModel, EmailStr, AnyUrl, Field # use to do the type validation and data validation 
from typing import List, Dict, Optional, Annotated # it is used to check the type within the type also

class Patient(BaseModel):

    name: Annotated[str, Field(max_length=50, title='Name of the patient', description='Give the name of the patient in less than 50 chars', examples=['vamsi', 'Hareesh'])]
    email: EmailStr
    linkedin_url: AnyUrl
    age: int = Field(gt=0, lt=120) # here we doing the data validation and type checking also
    weight: Annotated[float, Field(gt=0, strict=True)]
    married: Annotated[bool, Field(default=None, description='Is the patient married or not')]
    allergies: Annotated[Optional[List[str]], Field(default=None, max_length=5)] 
    contact_details: Dict[str, str]


def update_patient_data(patient: Patient):

    print(patient.name)
    print(patient.age)
    print(patient.allergies)
    print(patient.married)
    print('updated')

patient_info = {'name':'Hareesh', 'email':'abc@gmail.com', 'linkedin_url':'http://linkedin.com/1322', 'age': '21', 'weight': 57,'contact_details':{'phone':'2353462'}}

patient1 = Patient(**patient_info)

update_patient_data(patient1)

print(patient1)


Hareesh
21
None
None
updated
name='Hareesh' email='abc@gmail.com' linkedin_url=AnyUrl('http://linkedin.com/1322') age=21 weight=57.0 married=None allergies=None contact_details={'phone': '2353462'}


# 2

In [2]:
from pydantic import BaseModel, EmailStr, AnyUrl, Field, field_validator # if we need custom validation of the data 
from typing import List, Dict, Optional, Annotated

class Patient(BaseModel):

    name: str
    email: EmailStr
    age: int
    weight: float
    married: bool
    allergies: List[str]
    contact_details: Dict[str, str]

    @field_validator('email')
    @classmethod
    def email_validator(cls, value):

        valid_domains = ['hdfc.com', 'icici.com']
        # abc@gmail.com
        domain_name = value.split('@')[-1]

        if domain_name not in valid_domains:
            raise ValueError('Not a valid domain')

        return value
    
    @field_validator('name')
    @classmethod
    def transform_name(cls, value):
        return value.upper()
    
    @field_validator('age', mode='after') # this mode = after makes sure that age value got here after type conversion
    @classmethod
    def validate_age(cls, value):
        if 0 < value < 100:
            return value
        else:
            raise ValueError('Age should be in between 0 and 100')


def update_patient_data(patient: Patient):

    print(patient.name)
    print(patient.age)
    print(patient.allergies)
    print(patient.married)
    print('updated')

patient_info = {'name':'Hareesh', 'email':'abc@icici.com', 'age': '21', 'weight': 57, 'married': False, 'allergies': ['pollen', 'dust'], 'contact_details':{'phone':'2353462'}}

patient1 = Patient(**patient_info) # validation -> type coercion

update_patient_data(patient1)


HAREESH
21
['pollen', 'dust']
False
updated


# 3

In [3]:
from pydantic import BaseModel, EmailStr, model_validator
from typing import List, Dict

class Patient(BaseModel):
    name: str
    email: EmailStr
    age: int
    weight: float
    married: bool
    allergies: List[str]
    contact_details: Dict[str, str]

    @model_validator(mode='after')
    def validate_emergency_contact(self):
        if self.age > 60 and 'emergency' not in self.contact_details:
            raise ValueError('Patients older than 60 must have an emergency contact')
        return self

def update_patient_data(patient: Patient):
    print(patient.name)
    print(patient.age)
    print(patient.allergies)
    print(patient.married)
    print('updated')

patient_info = {
    'name': 'Hareesh',
    'email': 'abc@icici.com',
    'age': 23,
    'weight': 57,
    'married': False,
    'allergies': ['pollen', 'dust'],
    'contact_details': {'phone': '2353462', 'emergency': '235236'}
}

patient1 = Patient(**patient_info)
update_patient_data(patient1)


Hareesh
23
['pollen', 'dust']
False
updated


Perfect question — let’s **trace the full execution flow** of what happens internally when you do:

```python
patient1 = Patient(**patient_info)
```

with your corrected **Pydantic v2** model 👇

---

## ⚙️ Step-by-Step Execution Flow

### **1️⃣ Class Definition Stage**

When Python runs your class definition:

```python
class Patient(BaseModel):
    ...
    @model_validator(mode='after')
    def validate_emergency_contact(self):
        ...
```

* Pydantic sees `Patient` inherits from `BaseModel`.
* It **parses the field annotations** (`name: str`, `email: EmailStr`, etc.) and builds a **validation schema**.
* It also registers your method `validate_emergency_contact()` as a **model-level validator** that should run **after** all field values have been validated and assigned.

✅ So before you ever create an object, Pydantic has already built a "blueprint" of how validation should happen.

---

### **2️⃣ Object Instantiation Begins**

When you call:

```python
patient1 = Patient(**patient_info)
```

Python does this internally:

* Calls the **`__init__()`** method inherited from `BaseModel`.
* Passes your dictionary (`patient_info`) to Pydantic’s internal validation engine.

---

### **3️⃣ Core Validation Phase**

Pydantic now runs the **core validation** pipeline:

1. **Checks each field** according to its type annotation.
2. Converts or raises errors as needed.

Let’s go field by field:

| Field             | Expected Type    | Provided                           | Validation                  |
| ----------------- | ---------------- | ---------------------------------- | --------------------------- |
| `name`            | `str`            | `'Hareesh'`                        | OK ✅                        |
| `email`           | `EmailStr`       | `'abc@icici.com'`                  | Validated as proper email ✅ |
| `age`             | `int`            | `23`                               | OK ✅                        |
| `weight`          | `float`          | `57` (int → float)                 | Auto-converted ✅            |
| `married`         | `bool`           | `False`                            | OK ✅                        |
| `allergies`       | `List[str]`      | `['pollen', 'dust']`               | Each element validated ✅    |
| `contact_details` | `Dict[str, str]` | `{'phone': ..., 'emergency': ...}` | Keys/values validated ✅     |

After all field checks, Pydantic constructs a **temporary model instance** with these validated values.

---

### **4️⃣ Model-Level Validation Phase**

Now Pydantic sees that your model has a **`@model_validator(mode='after')`**, so it executes it **after** all field values are set.

It calls your function:

```python
def validate_emergency_contact(self):
```

* Here, `self` is **the fully constructed `Patient` instance** (already has all attributes validated).
* You can now check **relationships between fields**, not just individual fields.

So it runs:

```python
if self.age > 60 and 'emergency' not in self.contact_details:
    raise ValueError('Patients older than 60 must have an emergency contact')
```

---

### **5️⃣ Validator Decision**

* For your data, `self.age = 23`, so the condition `self.age > 60` is **False**.
* Hence, no error is raised.
* The validator **returns `self`** (the validated model).

---

### **6️⃣ Model Finalization**

Pydantic accepts the validated model and finishes instantiation:

```python
patient1 = <Patient(name='Hareesh', email='abc@icici.com', age=23, ... )>
```

Now `patient1` is a fully validated Pydantic object.

---

### **7️⃣ Function Call**

Then you call:

```python
update_patient_data(patient1)
```

* `patient1` (a `Patient` instance) is passed to your function.
* Inside, you print some of its attributes:

  ```
  Hareesh
  23
  ['pollen', 'dust']
  False
  updated
  ```

---

## 🔁 Summary of the Entire Lifecycle

| Stage                         | What Happens                                  | Who Handles It    |
| ----------------------------- | --------------------------------------------- | ----------------- |
| **Class Definition**          | Pydantic analyzes fields and validators       | Python + Pydantic |
| **Instantiation Call**        | `Patient(**data)`                             | Python            |
| **Field Validation**          | Each field’s type and constraints are checked | Pydantic          |
| **Model Validator Execution** | Runs `validate_emergency_contact(self)`       | Pydantic          |
| **Model Creation**            | Returns a valid `Patient` instance            | Pydantic          |
| **Use in Code**               | You access and use attributes safely          | You / Your Code   |

---

### 🧠 Extra Insight: What Happens If Validation Fails

Example:

```python
patient_info['age'] = 65
patient_info['contact_details'] = {'phone': '12345'}
patient1 = Patient(**patient_info)
```

Then during Step 4 (model-level validation):

```python
if self.age > 60 and 'emergency' not in self.contact_details:
    raise ValueError(...)
```

is `True`, so Pydantic raises a **`ValidationError`**, wrapping your ValueError into a structured error message:

```
pydantic_core._pydantic_core.ValidationError: 1 validation error for Patient
  Patients older than 60 must have an emergency contact (type=value_error)
```

and no object is created.

---

### ✅ Final Flow Summary Diagram

```
User Code: Patient(**data)
       │
       ▼
Pydantic: Validate fields (types, constraints)
       │
       ▼
Pydantic: Run @model_validator(mode='after')
       │
       ├── Pass → Return model instance
       └── Fail → Raise ValidationError
       │
       ▼
User Code: Receives validated Patient object
```

---

Would you like me to also show **how the flow differs between `field_validator` and `model_validator`** (so you understand when to use each)?


# 4

In [1]:
from pydantic import BaseModel, EmailStr, computed_field
from typing import List, Dict

class Patient(BaseModel):

    name: str
    email: EmailStr
    age: int
    weight: float # kg
    height: float # mtr
    married: bool
    allergies: List[str]
    contact_details: Dict[str, str]

    @computed_field
    @property
    def bmi(self) -> float:
        bmi = round(self.weight/(self.height**2),2)
        return bmi



def update_patient_data(patient: Patient):

    print(patient.name)
    print(patient.age)
    print(patient.allergies)
    print(patient.married)
    print('BMI', patient.bmi)
    print('updated')

patient_info = {'name':'Hareesh', 'email':'abc@icici.com', 'age': '65', 'weight': 53, 'height': 1.72, 'married': False, 'allergies': ['pollen', 'dust'], 'contact_details':{'phone':'2353462', 'emergency':'235236'}}

patient1 = Patient(**patient_info) 

update_patient_data(patient1)


Hareesh
65
['pollen', 'dust']
False
BMI 17.92
updated


Absolutely 👍 — here’s a **simple, clean markdown-style note** you can directly copy into your documentation or learning notes:

---

## 🧩 `@property` and `@computed_field` in Pydantic

### 🔹 `@property` (Python built-in)

* Used to **create read-only or computed attributes** in a class.
* Lets you **access a method like an attribute**, without parentheses.
* Runs a **function each time** you access it.

**Example:**

```python
class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year

    @property
    def age(self):
        return 2025 - self.birth_year

p = Person("John", 2000)
print(p.age)   # No () — calls function automatically → 25
```

**Key Points:**

* `@property` makes `obj.method()` behave like `obj.attribute`.
* Used when a value depends on other attributes.
* Value is **not stored**, it’s **computed on access**.
* Only exists in memory, **not part of Pydantic model schema**.

---

### 🔹 `@computed_field` (Pydantic v2 feature)

* Used to **turn a computed property into a real Pydantic field**.
* Combines the behavior of `@property` + adds Pydantic integration.
* Makes computed values **included in `.model_dump()`, `.model_json_schema()`, and FastAPI responses**.

**Example:**

```python
from pydantic import BaseModel, computed_field

class Patient(BaseModel):
    weight: float
    height: float

    @computed_field
    @property
    def bmi(self) -> float:
        return round(self.weight / (self.height ** 2), 2)

p = Patient(weight=75, height=1.75)
print(p.bmi)              # 24.49 (computed dynamically)
print(p.model_dump())     # {'weight': 75, 'height': 1.75, 'bmi': 24.49}
```

**Key Points:**

* Runs automatically when you access it.
* **Included** in model output and schema.
* Perfect for **derived fields** (like BMI, total_price, age, etc.).
* Requires both `@computed_field` and `@property` decorators.

---

### 🧠 Summary Table

| Feature                                | `@property` | `@computed_field` |
| -------------------------------------- | ----------- | ----------------- |
| Computed dynamically                   | ✅           | ✅                 |
| Used like an attribute                 | ✅           | ✅                 |
| Included in `.model_dump()`            | ❌           | ✅                 |
| Included in JSON Schema / FastAPI docs | ❌           | ✅                 |
| Part of Pydantic validation            | ❌           | ✅                 |
| Available in Pydantic v1               | ✅           | ❌ (only v2+)      |

---

**In short:**

> `@property` = Python-only computed value.
> `@computed_field` = Pydantic-aware computed value that behaves like a field.

---

Would you like me to add a small “when to use” section (like short best-practice tips) for your notes too?


# 5

In [3]:
from pydantic import BaseModel

class Address(BaseModel):

    city: str
    state: str
    pin: str

class Patient(BaseModel):

    name: str
    gender: str = 'Male'
    age: int
    address: Address

address_dict = {'city': 'gurgaon', 'state': 'haryana', 'pin': '122001'}

address1 = Address(**address_dict)

patient_dict = {'name': 'nitish', 'age': 35, 'address': address1}

patient1 = Patient(**patient_dict)

temp = patient1.model_dump(exclude_unset=True) # include = ['name'] only name we get and exclue also works
# exclude_unset True do if we are not specifyin the during the creation default values are assigned thos default assignedd properties
# are not returned
temp2 = patient1.model_dump_json()

print(temp)
print(type(temp))

print(temp2)


{'name': 'nitish', 'age': 35, 'address': {'city': 'gurgaon', 'state': 'haryana', 'pin': '122001'}}
<class 'dict'>
{"name":"nitish","gender":"Male","age":35,"address":{"city":"gurgaon","state":"haryana","pin":"122001"}}


### Automatic Validation — “No Extra Work Needed”

When we say **“automatically—no extra work needed,”** it refers to how **Pydantic handles validation of nested models** without requiring any manual checks or custom code.

---

### What It Means

* Pydantic automatically validates **all nested models** when you create an instance of a parent model.
* You don’t have to manually call or check validation for inner models.
* Validation happens **recursively**, meaning every nested level is checked according to its defined types and constraints.

---

### Example

```python
from pydantic import BaseModel

class Address(BaseModel):
    city: str
    state: str
    pin: str

class Patient(BaseModel):
    name: str
    age: int
    address: Address

# Passing nested data
data = {
    'name': 'Ravi',
    'age': 30,
    'address': {'city': 'Delhi', 'state': 'Delhi', 'pin': '110001'}
}

patient = Patient(**data)
```

Here:

* `Patient` contains an inner model `Address`.
* When we pass the dictionary for `address`, Pydantic automatically:

  1. Converts it into an `Address` object.
  2. Validates each field inside it (`city`, `state`, `pin`).

You don’t have to manually create `Address` or check for missing fields.

---

### Why It’s Useful

* **Saves time:** No manual validation or nested parsing required.
* **Ensures consistency:** All data at every level follows the same validation rules.
* **Improves reliability:** Even deeply nested data is guaranteed to be valid.

---

### Summary

Automatic validation in Pydantic means:

> Once you define your models (including nested ones),
> Pydantic automatically validates **everything inside them**
> whenever an object is created — **no extra validation code is needed.**
